In [1]:
import numpy as np
import pandas as pd
import re
import string
import nltk
from nltk.corpus import stopwords
import torch
import torch.nn.functional as F
from transformers import AutoTokenizer, AutoModel
from tqdm import tqdm

nltk.download('stopwords')

[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\Nikita\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


True

In [2]:
class DataPreprocessor:
    """
    Класс для вспомогательных функций преобразования данных
    """
    def __init__(self):
        self.stop_words = set(stopwords.words('russian')) | {'здравствуйте'}
        self.translit_map = {
            'alfabank': 'альфабанк',
            'actions': 'акции',
            'rules': 'правила',
            'alfafuture': 'альфа будущее',
        }

    def clean_text(self, text: str) -> str:
        """Очистка текста"""
        if pd.isna(text):
            return ""

        text = str(text).lower()
        text = re.sub(r'\d+', '', text)
        text = re.sub(f'[{re.escape(string.punctuation)}]', '', text)
        text = re.sub(r'[—«»]', '', text)
        text = re.sub(r'\n', ' ', text)
        text = text.replace('ё', 'е')

        text = ' '.join([word for word in text.split() if word not in self.stop_words and len(word) > 2])
        return text

    def translate_translit(self, text: str) -> str:
        """Преобразование транслита в кириллицу где возможно"""
        for eng, rus in self.translit_map.items():
            text = text.replace(eng, rus)
        return text

    def websites_url_splitting(self, url: str) -> str:
        """Разбиение URL на части с улучшенной обработкой"""
        if pd.isna(url):
            return ""

        url_parts = url.split('/')[3:]
        filtered_parts = [part for part in url_parts if 3 < len(part) < 50 and not part.startswith('2')]

        processed_parts = []
        for part in filtered_parts:
            part = part.replace('-', ' ')
            part = self.translate_translit(part)
            processed_parts.append(part)

        return ' '.join(processed_parts)

    def chunk_by_words(self, text):
        """Разбиение на два или три слова"""
        words = str(text).strip().split()
        
        if len(words) <= 3:
            return [' '.join(words)] if words else []
        
        chunks = []
        
        if len(words) % 2 == 0:
            # Для четного количества - просто разбиваем на пары
            chunks = [' '.join(words[i:i+2]) for i in range(0, len(words), 2)]
        else:
            # Для нечетного количества
            if len(words) % 3 == 0:
                # Если делится на 3 без остатка
                chunks = [' '.join(words[i:i+3]) for i in range(0, len(words), 3)]
            else:
                # Разбиваем на тройки, оставляя 4 или 2 слова в конце
                triplets_count = len(words) // 3
                if len(words) % 3 == 1:
                    # Осталось 1 слово - уменьшаем количество троек на 1
                    triplets_count -= 1
                
                # Добавляем тройки
                for i in range(0, triplets_count * 3, 3):
                    chunks.append(' '.join(words[i:i+3]))
                
                # Обрабатываем оставшиеся слова
                remaining = words[triplets_count * 3:]
                
                if len(remaining) == 4:
                    chunks.append(' '.join(remaining[:2]))
                    chunks.append(' '.join(remaining[2:]))
                elif len(remaining) == 2:
                    chunks.append(' '.join(remaining))
                elif len(remaining) == 1:
                    # Добавляем к последнему чанку
                    chunks[-1] = chunks[-1] + ' ' + remaining[0]
        
        return chunks

In [3]:
class DataTransformer(DataPreprocessor):
    """
    Класс для преобразования данных
    """
    def __init__(self, questions_df: pd.DataFrame, websites_df: pd.DataFrame):
        super().__init__()
        self.questions_df = questions_df.copy()
        self.websites_df = websites_df.copy()

    def transform_questions(self) -> pd.DataFrame:
        """Трансформация вопросов"""
        self.questions_df['query_cleaned'] = self.questions_df['query'].apply(self.clean_text).apply(lambda x: self.chunk_by_words(x))
        return self.questions_df
    
    def transform_websites(self) -> pd.DataFrame:
        """Трансформация веб-сайтов"""
        self.websites_df['title_cleaned'] = self.websites_df['title'].apply(self.clean_text).apply(lambda x: self.chunk_by_words(x))
        self.websites_df['text_cleaned'] = self.websites_df['text'].apply(self.clean_text).apply(lambda x: self.chunk_by_words(x))
        self.websites_df['url_cleaned'] = self.websites_df['url'].apply(self.websites_url_splitting).apply(lambda x: self.chunk_by_words(x))
    
        self.websites_df['website_all_data'] = self.websites_df['title_cleaned'] + self.websites_df['text_cleaned'] + self.websites_df['url_cleaned']
    
        return self.websites_df

In [4]:
def pool(hidden_state, mask, pooling_method="cls"):
    """Пулинг эмбеддингов"""
    if pooling_method == "mean":
        s = torch.sum(hidden_state * mask.unsqueeze(-1).float(), dim=1)
        d = mask.sum(axis=1, keepdim=True).float()
        return s / d
    elif pooling_method == "cls":
        return hidden_state[:, 0]
    elif pooling_method == "max":
        return torch.max(hidden_state * mask.unsqueeze(-1).float(), dim=1)[0]
        
class TransformersPredictionPipeline:
    """Класс с моделью на основе transformers"""
    def __init__(self, model_name="ai-forever/ru-en-RoSBERTa", pooling_method="cls"):
        print(f"Загрузка модели: {model_name}")
        self.tokenizer = AutoTokenizer.from_pretrained(model_name)
        self.model = AutoModel.from_pretrained(model_name)
        self.pooling_method = pooling_method
        self.model.eval()  # Переводим модель в режим оценки
        print("Модель загружена!")

    def encode_texts(self, texts: list) -> torch.Tensor:
        """Кодирование списка текстов в эмбеддинги"""
        print(f"Кодируем {len(texts)} текстов...")

        # Токенизация
        tokenized_inputs = self.tokenizer(
            texts,
            max_length=512,
            padding=True,
            truncation=True,
            return_tensors="pt"
        )

        # Получение эмбеддингов
        with torch.no_grad():
            outputs = self.model(**tokenized_inputs)

        # Применяем пулинг
        embeddings = pool(
            outputs.last_hidden_state,
            tokenized_inputs["attention_mask"],
            pooling_method=self.pooling_method
        )

        # Нормализуем эмбеддинги
        embeddings = F.normalize(embeddings, p=2, dim=1)

        return embeddings

    def fit_predict_pairs(self, questions_df: pd.DataFrame, websites_df: pd.DataFrame,
                          query_column='query_cleaned',
                          website_column='website_all_data',
                          k=5) -> list:

        print("Кодирование текстов и вычисление схожести...")

        # Получаем тексты для кодирования
        queries = questions_df[query_column].tolist()
        websites_texts = websites_df[website_column].tolist()

        # Сохраняем ID и URL для результатов
        self.question_ids = questions_df['q_id'].tolist()
        self.website_ids = websites_df['web_id'].tolist()
        self.website_urls = websites_df['url'].tolist()

        print(f"Кодируем {len(queries)} вопросов...")
        questions_emb = self.encode_texts(queries)

        print(f"Кодируем {len(websites_texts)} сайтов...")
        websites_emb = self.encode_texts(websites_texts)

        print("Вычисляем схожесть...")

        # Вычисляем косинусную схожесть
        scores = questions_emb @ websites_emb.T  # Матрица схожести

        # Обрабатываем результаты
        results = []
        for i, query in enumerate(queries):
            query_scores = scores[i].cpu().tolist()

            # Комбинируем веб-сайты, URLs и scores
            doc_score_pairs = list(zip(self.website_ids, self.website_urls, websites_texts, query_scores))

            # Сортируем по убыванию score
            doc_score_pairs = sorted(doc_score_pairs, key=lambda x: x[3], reverse=True)

            # Сохраняем топ-N результатов
            top_results = []
            for web_id, url, doc_text, score in doc_score_pairs[:k]:
                top_results.append({
                    'web_id': web_id,
                    'url': url,
                    'score': score,
                    'text': doc_text[:200] + '...' if len(doc_text) > 200 else doc_text
                })

            results.append({
                'q_id': self.question_ids[i],
                'query': query,
                'web_list': top_results
            })

        print("Готово!")
        return results

    def fit_predict_pairs_advanced(self, questions_df: pd.DataFrame, websites_df: pd.DataFrame,
                                 k=5, weights=(0.5, 0.3, 0.2)) -> list:
        """
        Расширенный метод с раздельным кодированием разных типов текста
        """
        print("Расширенное кодирование текстов...")

        queries = questions_df['query_cleaned'].tolist()

        # Разделяем данные веб-сайтов
        titles = websites_df['title_cleaned'].tolist()
        url_texts = websites_df['url_cleaned'].tolist()
        main_texts = websites_df['text_cleaned'].tolist()

        self.question_ids = questions_df['q_id'].tolist()
        self.website_ids = websites_df['web_id'].tolist()
        self.website_urls = websites_df['url'].tolist()

        print("Кодируем разные типы данных...")

        # Кодируем раздельно
        queries_emb = self.encode_texts(queries)
        titles_emb = self.encode_texts(titles)
        urls_emb = self.encode_texts(url_texts)
        texts_emb = self.encode_texts(main_texts)

        print("Вычисляем комбинированную схожесть...")

        title_weight, url_weight, text_weight = weights

        results = []
        for i, query in enumerate(queries):
            # Вычисляем схожести для разных типов данных
            title_scores = (queries_emb[i] @ titles_emb.T).cpu().tolist()
            url_scores = (queries_emb[i] @ urls_emb.T).cpu().tolist()
            text_scores = (queries_emb[i] @ texts_emb.T).cpu().tolist()

            # Комбинируем scores с весами
            combined_scores = []
            for j in range(len(title_scores)):
                combined_score = (title_scores[j] * title_weight +
                                url_scores[j] * url_weight +
                                text_scores[j] * text_weight)
                combined_scores.append(combined_score)

            # Формируем результаты
            doc_score_pairs = list(zip(self.website_ids, self.website_urls,
                                     titles, combined_scores))
            doc_score_pairs = sorted(doc_score_pairs, key=lambda x: x[3], reverse=True)

            top_results = []
            for web_id, url, title, score in doc_score_pairs[:k]:
                top_results.append({
                    'web_id': web_id,
                    'url': url,
                    'score': score,
                    'title': title
                })

            results.append({
                'q_id': self.question_ids[i],
                'query': query,
                'web_list': top_results
            })

        return results

    def get_top_k_dataframe(self, pairs_results: list = None, k: int = 5) -> pd.DataFrame:
        """
        Создает датафрейм с id вопроса и топ-k наиболее подходящих ссылок
        """
        results = []
        for pair_result in pairs_results:
            q_id = pair_result['q_id']
            top_websites = pair_result['web_list'][:k]

            urls = [website['web_id'] for website in top_websites]

            results.append({
                'q_id': q_id,
                'web_list': urls
            })

        return pd.DataFrame(results)

In [5]:
# Загрузка данных
questions = pd.read_csv('questions_clean.csv', delimiter=',', encoding='utf-8')[:20]
websites = pd.read_csv('websites_updated.csv', delimiter=',', encoding='utf-8')[:20]
websites = websites.dropna(axis=0)

In [6]:
questions.head()

,q_id,query
0,1,Номер счета
1,2,Где узнать бик и счёт
2,3,Мне не приходят коды для подтверждения данной ...
3,4,"Оформила рассрочку ,но уведомлений никаких не ..."
4,5,"Здравствуйте, когда смогу пользоваться кредитн..."


In [7]:
websites.head()

,web_id,url,kind,title,text
0,1,https://alfabank.ru/,html,"Альфа-Банк - кредитные и дебетовые карты, кред...",Рассчитайте выгоду\nРасчёт калькулятора предва...
1,2,https://alfabank.ru/a-club/,html,А-Клуб. Деньги имеют значение,Брокерские услуги\nОткрытие брокерского счёта ...
2,3,https://alfabank.ru/a-club/ultimate/,html,А-Клуб. Деньги имеют значение,Хотите получить больше информации?\nПозвоните ...
3,4,https://alfabank.ru/actions/rules/,html,Скидки по картам,Правила проведения Акции «Альфа Пятница. Бараб...
4,5,https://alfabank.ru/alfafuture/,html,Альфа‑Будущее: Платформа для развития студенто...,Образование\nМагистратуры\nМагистратура ВШЭ\nМ...


In [8]:
# Преобразование данных
transformer = DataTransformer(questions, websites)
questions_cleaned = transformer.transform_questions()
websites_cleaned = transformer.transform_websites()

In [9]:
questions_cleaned.head()

,q_id,query,query_cleaned
0,1,Номер счета,[номер счета]
1,2,Где узнать бик и счёт,[узнать бик счет]
2,3,Мне не приходят коды для подтверждения данной ...,"[приходят коды подтверждения, данной операции]"
3,4,"Оформила рассрочку ,но уведомлений никаких не ...","[оформила рассрочку уведомлений, никаких пришло]"
4,5,"Здравствуйте, когда смогу пользоваться кредитн...","[смогу пользоваться, кредитной картой]"


In [10]:
websites_cleaned.head()

,web_id,url,kind,title,text,title_cleaned,text_cleaned,url_cleaned,website_all_data
0,1,https://alfabank.ru/,html,"Альфа-Банк - кредитные и дебетовые карты, кред...",Рассчитайте выгоду\nРасчёт калькулятора предва...,"[альфабанк кредитные дебетовые, карты кредиты ...","[рассчитайте выгоду расчет, калькулятора предв...",[],"[альфабанк кредитные дебетовые, карты кредиты ..."
1,2,https://alfabank.ru/a-club/,html,А-Клуб. Деньги имеют значение,Брокерские услуги\nОткрытие брокерского счёта ...,"[аклуб деньги, имеют значение]","[брокерские услуги, открытие брокерского, счет...",[a club],"[аклуб деньги, имеют значение, брокерские услу..."
2,3,https://alfabank.ru/a-club/ultimate/,html,А-Клуб. Деньги имеют значение,Хотите получить больше информации?\nПозвоните ...,"[аклуб деньги, имеют значение]","[хотите получить информации, позвоните нам отв...",[a club ultimate],"[аклуб деньги, имеют значение, хотите получить..."
3,4,https://alfabank.ru/actions/rules/,html,Скидки по картам,Правила проведения Акции «Альфа Пятница. Бараб...,[скидки картам],"[правила проведения акции, альфа пятница бараб...",[акции правила],"[скидки картам, правила проведения акции, альф..."
4,5,https://alfabank.ru/alfafuture/,html,Альфа‑Будущее: Платформа для развития студенто...,Образование\nМагистратуры\nМагистратура ВШЭ\nМ...,"[альфа‑будущее платформа развития, студентов у...","[образование магистратуры, магистратура вшэ, м...",[альфа будущее],"[альфа‑будущее платформа развития, студентов у..."


In [11]:
websites_cleaned['website_all_data'].tolist()

[['альфабанк кредитные дебетовые',
  'карты кредиты наличными',
  'автокредитование ипотека другие',
  'банковские услуги физическим',
  'юридическим лицам альфабанк',
  'рассчитайте выгоду расчет',
  'калькулятора предварительный персональные',
  'условия сможете узнать',
  'оформления заявки получайте',
  'альфабанком сервисы курсы',
  'валют валюта покупка',
  'продажа доллар сша',
  'usd евро eur',
  'китайский юань cny',
  'валют информация курсах',
  'обмена иностранных валют',
  'является справочной меняться',
  'течение дня точную',
  'информацию курсах узнать',
  'интернетбанке список отделений',
  'доступен ссылке'],
 ['аклуб деньги',
  'имеют значение',
  'брокерские услуги',
  'открытие брокерского',
  'счета приложении',
  'альфа‑инвестиции расширенный',
  'инструментарий биржевая',
  'внебиржевая маржинальная',
  'торговля биржевые',
  'структурные облигации',
  'первичные размещения',
  'инвестиционное консультирование',
  'инвесторов брокерским',
  'счетом альфа‑инвести

In [ ]:
# Создание предсказаний с transformers
pipeline = TransformersPredictionPipeline(pooling_method="cls")  # или "mean"

# Базовый подход
pairs_results = pipeline.fit_predict_pairs(questions_cleaned, websites_cleaned, k=5)

# Или расширенный подход
# pairs_results = pipeline.fit_predict_pairs_advanced(
#     questions_cleaned, websites_cleaned, k=5, weights=(0.5, 0.3, 0.2)
# )

# Создание финального датафрейма
result_df = pipeline.get_top_k_dataframe(pairs_results, k=5)

# Сохранение результатов
result_df.to_csv('submit.csv', index=False)

print("Результаты сохранены!")
print(result_df.head())

Загрузка модели: ai-forever/ru-en-RoSBERTa


model.safetensors:   0%|          | 0.00/1.61G [00:00<?, ?B/s]